In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.spatial.transform import Rotation as R
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# goal position
xg, yg, zg = 1150, 200, 300 #TODO find reasonable numbers
x = np.array([xg,yg,zg])

In [3]:
# robot linkages defn, lengths in mm
l0, l1, l2, l3, l4, l5, l6, lg = \
    81, 192.5, 400, 168.5, 400, 136.3, 133, 20 # unknown lg value, TODO measure

In [4]:
# end effector (w/o gripper) location. 317 is default height of J1
lx = l0+l2+l4+l6
ly = l1+l3+l5
lz = 317
print(f'{lx} {ly} {lz}')
l = np.array([lx, ly, lz])

1014 497.3 317


In [5]:
# calculate angle to minimize y-diff
# theta_0 = np.arctan2((yg-ly), (xg-lx)) # FIGURE OUT
theta_0 = np.arctan2(xg, -yg) + np.arccos(ly/np.sqrt(xg**2 + yg**2)) - np.pi # from sortersawyer
print(f'{theta_0}')

-0.2679201204199728


In [6]:
# Verification
r = R.from_euler('z', theta_0).as_matrix()

In [7]:
l_transformed = r @ l

In [8]:
fig = go.Figure()

In [9]:
fig.add_trace(go.Scatter(x=[0,l_transformed[0]], y=[0,l_transformed[1]], mode='lines', name='endrot'))
fig.add_trace(go.Scatter(x=[0,lx], y=[0,ly], mode='lines', name='init_l6'))
fig.add_trace(go.Scatter(x=[xg], y=[yg], mode='markers', name='goal'))

found theta_0

In [10]:
l_transformed

array([1109.47251658,  211.12561418,  317.        ])

In [11]:
# coordinates of the beginning of j3, end of l2
j3 = np.array([l0 + l2, l1, lz])

In [12]:
j3_t = r @ j3

In [13]:
j3_t

array([514.79946155,  58.29892272, 317.        ])

In [14]:
# position of ball release given theta 3, theta_5
xr = lambda theta_3, theta_5: j3_t[0] + l4*np.cos(theta_3) + (l6+lg)*np.cos(theta_3+theta_5)
zr = lambda theta_3, theta_5: j3_t[2] + l4*np.sin(theta_3) + (l6+lg)*np.sin(theta_3+theta_5)

In [15]:
w3, w5 = 1, 3 # arbitrarily

In [16]:
xr_dot = lambda theta_3, theta_5: l4*(-np.sin(theta_3))*w3 + (l6+lg)*(-np.sin(theta_3+theta_5))*(w3+w5)
zr_dot = lambda theta_3, theta_5: l4*(np.cos(theta_3))*w3 + (l6+lg)*(np.cos(theta_3+theta_5))*(w3+w5)

In [17]:
# parametrized with kinematics
x = lambda t, theta_3, theta_5: xr(theta_3, theta_5) + xr_dot(theta_3, theta_5)*t
z = lambda t, theta_3, theta_5: zr(theta_3, theta_5) + zr_dot(theta_3, theta_5)*t - (0.5)*(9.81*1000)*(t**2)

In [18]:
# what we are trying to optimize
def cost(q):
    t, theta_3, theta_5 = q[0], q[1], q[2]
    return (x(t, theta_3, theta_5) - xg)**2 + (z(t, theta_3, theta_5) - zg)**2


In [19]:
outp = minimize(cost, [100,0,0])

In [20]:
outp

      fun: 3104.8853628645797
 hess_inv: array([[ 1.20895006e-07, -1.31578319e-06,  2.92300717e-06],
       [-1.31578319e-06,  1.75589369e-05, -3.74413880e-05],
       [ 2.92300717e-06, -3.74413880e-05,  8.31691206e-05]])
      jac: array([ 0.00216675, -0.00027466, -0.00039673])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 428
      nit: 81
     njev: 106
   status: 2
  success: False
        x: array([-1.88159988e-01, -1.31754118e+04, -3.87605110e+04])